# **EDA for text_change variable**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
# feature_engineering
import re

from sklearn.preprocessing import SplineTransformer
# ploting
import matplotlib.pyplot as plt
# model initial:
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
#Set import folder
input_folder = "kaggle/input/linking-writing-processes-to-writing-quality/"

In [3]:
# Load train logs
train_logs = pd.read_csv(input_folder + "train_logs.csv",delimiter = ",",header = 0)
# Set the id and event id as index:
train_logs_indexed = train_logs.set_index(['id', 'event_id'])
# Load train scores
train_scores = pd.read_csv(input_folder +"train_scores.csv", delimiter = ",", header = 0)
scores = pd.Series(data = train_scores['score'].values, index = train_scores['id'].values, name = 'score')

In [4]:
train_logs.head(6)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1
5,001519c8,6,107296,107400,104,Input,q,q,q,3,1


In [5]:
train_logs.tail(3)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
8405895,fff05981,3617,2064657,2064765,108,Replace,q,q,q => q,1031,240
8405896,fff05981,3618,2069186,2069259,73,Nonproduction,Leftclick,Leftclick,NoChange,1028,240
8405897,fff05981,3619,2070065,2070133,68,Input,.,.,.,1029,240


In [6]:
train_logs.shape

(8405898, 11)

In [7]:
train_logs.text_change.describe()

count     8405898
unique       4111
top             q
freq      6128208
Name: text_change, dtype: object

In [8]:
score_60 = scores[scores == 6.0].index.values
score_05 = scores[scores == 0.5].index.values


In [9]:
train_logs.text_change.unique()[:25]

array(['NoChange', 'q', ' ', '.', ',', ' qqq qqqqq  =>  ',
       'qqqqq qq => q', 'qqqq  => q', '\n', "'", ';', 'qqqqqqq ', 'qq',
       'qqqqqqq qqqqqq  => q', 'qq qqqqqqqq qqqqq => q', 'qqqqq  => q',
       'qqqqqqqqq  => q', '-', '?', '"', '\n ', 'qqqqqq => q',
       'qqqqqqq qqq qqqq qqqq qqqq qq qqqqqqqq qqqqqqq  => q',
       'qqqqqqq  => q', 'qqqqq qqq qqqq qq '], dtype=object)

In [10]:
train_logs.text_change.unique().shape

(4111,)

In [11]:
# eda text_change
# Based on the above, discriminate if the changes has "?" and "=>"
def hasChar(x, character):
    out = 0 
    for strings in x:
        if character in strings:
            out = 1 
            break
    return out
def qCounter(x):
    h = " ".join(x)
    return np.log(len(re.findall(r" q ", h))+1)

def averageQ(x):
    h = " ".join(x)
    return np.log((len(re.findall(r" q ", h))+1)/(len(x) +1))
#test:
# statement = "The quick Brown Fox ?"
# print(hasChar(statement, "?"))
# v1 = 'qqqqqqq qqq qqqq qqqq qqqq qq qqqqqqqq qqqqqqq  => q'
# print(hasChar(v1, "=>"))

# del statement, v1
        
#execute now:
tc = train_logs.groupby(['id'])['text_change'].aggregate([
    ("tc_1", lambda x: hasChar(x,character = "?")), 
    ("tc_2", lambda x: hasChar(x,character = "=>")), 
    ("tc_3", lambda x: hasChar(x,character = "(")), 
    ("tc_4", lambda x: hasChar(x,character = "\"")), 
    ("tc_5", lambda x: hasChar(x,character = "-")), 
    ("tc_6", lambda x: qCounter(x)), 
    ("tc_7", lambda x: averageQ(x))]) 

tc.head(3)

,tc_1,tc_2,tc_3,tc_4,tc_5,tc_6,tc_7
id,,,,,,,
001519c8,0,1,0,0,0,6.967909,-0.879072
0022f953,1,1,0,1,1,6.854355,-0.951528
0042269b,0,1,0,1,1,7.488294,-0.839433


In [12]:
# EDA 2 for text change . Below is not implemented.
'''
Check the distribution of number of q's such as how many 2 qs, 3 qs a, 4 ... 8 qs being changed. Get the average
for each type of score and estimate the distribution. Compare this hypothesized distribution to each of the
participants and predict their score. 
'''
# check text changes per participant:

def text_change_distribution(v:str, max_word_length = 9):
    distribution_container = []
    start_flag = 1
    word_count = 0
    size = 0
    for i in range(1,max_word_length + 2):
        s = "q{%s} " % i
        f = re.findall(s, v)
        if(start_flag == 1):
            word_count = len(f)
            start_flag = 0
        else:
            size = word_count - len(f)
            distribution_container.append(size)
            word_count = len(f)
            
    return distribution_container
# Test if true:
v1 = 'qqqqqqq qqq qqqq qqqq qqqq qq qqqqqqqq qqqqqqq  => q'
v2 = 'qq qqqqqqqq qqqqqqq qq => q'
v3 = 'qq qq => q'
v4 = 'q qq => q'
v5 = 'q'
v6 = 'NoChange'
def test_func(b:bool):
    if(b == True):
        print("pass")
    else:
        print("fail")
# simple unit test for function above. Should return to True on all
print("test results:")
test_func(np.array_equal(text_change_distribution(v1), np.array([0, 1, 1, 3, 0, 0, 2, 1, 0])))
test_func(np.array_equal(text_change_distribution(v2), np.array([0, 2, 0, 0, 0, 0, 1, 1, 0])))
test_func(np.array_equal(text_change_distribution(v3), np.array([0, 2, 0, 0, 0, 0, 0, 0, 0])))
test_func(np.array_equal(text_change_distribution(v4), np.array([1, 1, 0, 0, 0, 0, 0, 0, 0])))
test_func(np.array_equal(text_change_distribution(v5), np.array([0, 0, 0, 0, 0, 0, 0, 0, 0])))
test_func(np.array_equal(text_change_distribution(v6), np.array([0, 0, 0, 0, 0, 0, 0, 0, 0])))


del v1, v2, v3, v4, v5, v6
# del text_change_distribution, test_func
# The code above is for reference only

test results:
pass
pass
pass
pass
pass
pass


In [13]:
pd.merge(tc, scores, left_index = True, right_index = True).corr()

,tc_1,tc_2,tc_3,tc_4,tc_5,tc_6,tc_7,score
tc_1,1.000000,0.114040,0.127081,0.227015,0.192336,0.294937,0.020982,0.284010
tc_2,0.114040,1.000000,0.085419,0.128934,0.217719,0.266352,-0.022849,0.249572
tc_3,0.127081,0.085419,1.000000,0.238456,0.243662,0.291353,0.016473,0.254369
tc_4,0.227015,0.128934,0.238456,1.000000,0.267089,0.278904,-0.000794,0.270913
tc_5,0.192336,0.217719,0.243662,0.267089,1.000000,0.398322,0.026392,0.397381
tc_6,0.294937,0.266352,0.291353,0.278904,0.398322,1.000000,0.142094,0.678699
tc_7,0.020982,-0.022849,0.016473,-0.000794,0.026392,0.142094,1.000000,0.101840
score,0.284010,0.249572,0.254369,0.270913,0.397381,0.678699,0.101840,1.000000


In [14]:
# New eda: Using Text Vectorizer
test_participant = train_logs_indexed.loc['001519c8'].copy()
test_participant2 = train_logs_indexed.loc['fff05981'].copy()

In [15]:
h1 = "".join(test_participant.text_change)
h2 = "".join(test_participant2.text_change)

In [16]:
print("Q in h1: {}".format(len(re.findall(r" q ", h1))))
print("Q in h2: {}".format(len(re.findall(r" q ", h2))))

Q in h1: 1
Q in h2: 38


In [17]:
print(len(re.findall(" q{2} ", h1)))

65


In [18]:
h3 = 'qq qqqq qqqqq qq qqqq qq qqq qqqqqq. qqqq qqq qqqq qqqqqqqq qq qqqqqqqqqq qqqqqqqqq qq qq qqq qqqqqqq qqqqqqqqqq qq qqq qqqqqqqqqq.'

In [19]:
text_change_distribution(h1, 12)

[3, 79, 53, 48, 41, 29, 24, 16, 20, 10, 6, 3]

In [20]:
text_change_distribution(h2, 12)

[49, 101, 71, 65, 42, 50, 38, 29, 12, 18, 20, 8]

In [21]:
sample = train_logs.groupby('id')['text_change'].aggregate(lambda x: text_change_distribution("".join(x), 12))


In [22]:
sample.head()

id
001519c8        [3, 79, 53, 48, 41, 29, 24, 16, 20, 10, 6, 3]
0022f953          [13, 70, 95, 62, 38, 33, 10, 7, 3, 5, 4, 6]
0042269b    [11, 106, 117, 106, 72, 48, 52, 31, 29, 12, 20...
0059420b          [13, 51, 36, 49, 26, 21, 25, 2, 3, 2, 2, 2]
0075873a       [21, 61, 69, 63, 41, 24, 14, 17, 11, 8, 10, 2]
Name: text_change, dtype: object

In [23]:
z1 = np.log(pd.DataFrame(np.stack(sample, axis = 0), index = sample.index)+1)

for i in np.arange(start = 0.1, stop = 4.5, step = 0.1):
    zout = pd.merge(z1, scores**i, left_index = True, right_index = True).corr()
    print("power {} = {}".format(i,zout['score'].sum()))

power 0.1 = 6.99247514102524
power 0.2 = 7.057635126811955
power 0.30000000000000004 = 7.115512928974851
power 0.4 = 7.166527371376991
power 0.5 = 7.211091948926925
power 0.6 = 7.249609850308636
power 0.7000000000000001 = 7.282470454157899
power 0.8 = 7.310046976582271
power 0.9 = 7.332695005376274
power 1.0 = 7.350751708585399
power 1.1 = 7.364535549844057
power 1.2000000000000002 = 7.37434637970191
power 1.3000000000000003 = 7.380465801564029
power 1.4000000000000001 = 7.3831577339625
power 1.5000000000000002 = 7.382669108818323
power 1.6 = 7.379230659229756
power 1.7000000000000002 = 7.3730577610602515
power 1.8000000000000003 = 7.364351300938452
power 1.9000000000000001 = 7.353298549797753
power 2.0 = 7.340074026207525
power 2.1 = 7.3248403378028675
power 2.2 = 7.307748992342763
power 2.3000000000000003 = 7.288941172492761
power 2.4000000000000004 = 7.268548470470457
power 2.5000000000000004 = 7.246693580309
power 2.6 = 7.223490946764634
power 2.7 = 7.199047370881063
power 2.800000

In [24]:
# Eda clas Part 2
class TextChangeTransformer2(BaseEstimator, TransformerMixin):
    def __init__(self, max_word_length, log_transform = True):
        self.max_word_length = max_word_length
        self.log_transform = log_transform

    def fit(self, X, y = None):
        return self
        
    def text_change_distribution(self, v:str):
        distribution_container = []
        start_flag = 1
        word_count = 0
        size = 0
        for i in range(1,self.max_word_length + 2):
            s = "q{%s} " % i
            f = re.findall(s, v)
            if(start_flag == 1):
                word_count = len(f)
                start_flag = 0
            else:
                size = word_count - len(f)
                distribution_container.append(size)
                word_count = len(f)
                
        return distribution_container
        
    def transform(self, X):
        
        X = X.groupby('id')['text_change'].aggregate(lambda r: self.text_change_distribution("".join(r)))
        X = pd.DataFrame(np.stack(X, axis = 0), index = X.index)
        if self.log_transform:
            X = np.log(X + 1)
        return X.values
        


In [25]:
'''
To check the number of qs for each participant and comparing this to the q distributions generated
'''
class TextChangeTransformer3(BaseEstimator, TransformerMixin):
    def __init__(self, scores, max_wl = 12):
        self.max_wl = max_wl
        self.scores = scores

    def fit(self, X, y = None):
        #Get the distribution
        X = X.groupby('id')['text_change'].aggregate(lambda r: self.text_change_distribution("".join(r)))
        self.column_names = []
        for n in range(1, self.max_wl+1):
            self.column_names.append('q_len={}'.format(n))
        X2 = pd.DataFrame(np.stack(X, axis = 0), index = X.index, columns = self.column_names)
        
        X2 = pd.merge(X2,scores, left_index = True, right_index = True)
        self.X2 = X2
        # The pop standard dev::
        self.prop_std = self.X2.groupby("score").std(ddof = 0)
        # The mean
        self.expected_count = self.X2.groupby("score").mean()
        

        return self
        
        
    def text_change_distribution(self, v:str):
        distribution_container = []
        start_flag = 1
        word_count = 0
        size = 0
        for i in range(1,self.max_wl + 2):
            s = "q{%s} " % i
            f = re.findall(s, v)
            if(start_flag == 1):
                word_count = len(f)
                start_flag = 0
            else:
                size = word_count - len(f)
                distribution_container.append(size)
                word_count = len(f)
                
        return distribution_container 


    def transform(self, X):
        print("Started TextChangeTransformer 3 Feature Engineering")
        grouped_X = X.groupby('id')['text_change'].aggregate(lambda r: self.text_change_distribution("".join(r)))
        grouped_X = pd.DataFrame(np.stack(grouped_X, axis = 0), index = grouped_X.index, columns = self.column_names)

        predictions = np.zeros(shape = grouped_X.shape)

        # Iterate in columns:
        for j in range(self.max_wl):
            mean = self.expected_count.iloc[:,j].copy()
            sd = self.prop_std.iloc[:,j].copy()
            
            for i in range(grouped_X.shape[0]):
                x_value = grouped_X.iloc[i,j]
                z_scores = (x_value - mean)/sd
                z_scores = np.abs(z_scores)
                
                min_values = np.partition(z_scores, 1)[0:3] 
                idx_0 = np.where(z_scores == min_values[0])[0][0]
                idx_1 = np.where(z_scores == min_values[1])[0][0]
                idx_2 = np.where(z_scores == min_values[2])[0][0]
                # mean or sd index is ok:
                predictions[i,j] = np.mean([sd.index[idx_0],sd.index[idx_1],sd.index[idx_2]]) 

        output = pd.DataFrame(predictions, index = grouped_X.index, columns = self.column_names)
        output['tc_ave'] = np.apply_along_axis(func1d = np.mean, axis = 1, arr = output)

        print("Done.. tc_3")
        return output.values 
        

In [26]:
tm = TextChangeTransformer3(scores, 15)
tm.fit(train_logs)
output = tm.transform(train_logs)

Started TextChangeTransformer 3 Feature Engineering
Done.. tc_3


In [27]:
output

array([[1.        , 2.66666667, 1.5       , ..., 1.        , 4.33333333,
        2.51111111],
       [1.5       , 2.33333333, 3.33333333, ..., 1.        , 1.        ,
        1.87777778],
       [1.        , 3.33333333, 3.66666667, ..., 1.5       , 3.66666667,
        3.54444444],
       ...,
       [1.        , 1.        , 1.5       , ..., 1.5       , 2.16666667,
        1.42222222],
       [1.66666667, 3.        , 3.        , ..., 4.33333333, 2.16666667,
        2.94444444],
       [4.33333333, 3.33333333, 2.66666667, ..., 4.33333333, 1.        ,
        3.42222222]])

In [28]:
LinearRegression().fit(output, scores).score(output,scores)

0.44522455787921067

In [29]:
# Eda TextChange 4
class TextChangeVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.vectorizer = CountVectorizer()

    def fit(self, X, y = None):
        feature = X.groupby('id')['text_change'].agg(lambda z: " ".join(z))
        self.vectorizer.fit(feature)
        return self


    def transform(self, X):
        feature = X.groupby('id')['text_change'].agg(lambda z: " ".join(z))
        output = self.vectorizer.transform(feature)
        return output 
        

In [30]:
v = TextChangeVectorizer()
v.fit(train_logs)
output = v.transform(train_logs)
output

<2471x70 sparse matrix of type '<class 'numpy.int64'>'
	with 11279 stored elements in Compressed Sparse Row format>